In [18]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, load_img, array_to_img, img_to_array
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense
from keras import applications
from keras import optimizers
import glob
import matplotlib.pyplot as plt
from skimage.io import imread
from keras.callbacks import ModelCheckpoint

In [31]:
# dimensions of our images.
img_width, img_height = 256, 256

weights_path = '../saved_weights/vgg16_weights.h5'
top_model_weights_path = '../saved_weights/bottleneck_fc_model_servet.h5'
train_data_dir = '../data/model/key19/train'
validation_data_dir = '../data/model/key19/test'
test_data_dir = '../data/model/key19/test'
nb_train_samples = int(len(glob.glob(train_data_dir+'/*/*')))
nb_validation_samples = int(len(glob.glob(validation_data_dir+'/*/*')))
nb_test_samples = int(len(glob.glob(test_data_dir+'/*/*')))
epochs = 2000
batch_size = 64
print('Found {} training samples, {} validation samples and {} test samples'.format(nb_train_samples,nb_validation_samples, nb_test_samples))

Found 448 training samples, 150 validation samples and 150 test samples


In [32]:
# number of files in each folder
nb_train_sub_samples = np.zeros(6)
nb_validation_sub_samples = np.zeros(6)
for i in range(1,7):
    path_train = train_data_dir+'/pos'+str(i)+'/*.png'
    path_validation = validation_data_dir+'/pos'+str(i)+'/*.png'
    nb_train_sub_samples[i-1] = int(len(glob.glob(path_train)))
    nb_validation_sub_samples[i-1] = int(len(glob.glob(path_validation)))

In [33]:
# build the VGG16 network
base_model = applications.VGG16(weights='imagenet', include_top=False,input_shape = (img_width, img_height, 3))
print('Model loaded.')

Model loaded.


In [34]:
for layer in base_model.layers[-4:]:
    layer.trainable = False
    
#base_model.summary()

In [35]:
# build a classifier model to put on top of the convolutional model
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(4096, activation='relu'))
top_model.add(Dense(6, activation='softmax'))

In [36]:
# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning
top_model.load_weights(top_model_weights_path)

OSError: Unable to open file (unable to open file: name = '../saved_weights/bottleneck_fc_model_servet.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
#top_model.summary()

In [ ]:
# add the model on top of the convolutional base
model = Model(inputs= base_model.input, outputs=top_model(base_model.output))

In [ ]:
model.summary()

In [ ]:
# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
#model.compile(loss='categorical_crossentropy', optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),metrics=['accuracy'])
#model.compile(loss='categorical_crossentropy', optimizer=optimizers.RMSprop(lr=1e-4, rho=0.9,epsilon=1e-08, decay=0.0))

from keras import metrics
model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0), metrics = [metrics.mae, metrics.categorical_accuracy])

In [10]:
train_datagen = ImageDataGenerator(#preprocessing_function=subtractmean,
                                    rotation_range=10,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    fill_mode='nearest',
                                    zca_whitening=True,
                                    featurewise_std_normalization = True,
                                    #horizontal_flip=True
)

validation_datagen = ImageDataGenerator(#preprocessing_function=subtractmean,
                                        #rotation_range=0,
                                        #width_shift_range=0,
                                        #height_shift_range=0,
                                        #shear_range=0,
                                        #zoom_range=0,
                                        #zca_whitening=False,
                                        #featurewise_std_normalization = True
)

In [11]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 293 images belonging to 6 classes.


In [12]:
validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 90 images belonging to 6 classes.


In [ ]:
checkpointer = ModelCheckpoint(filepath='../saved_weights/VGG16_key19_weights_servet.hdf5', verbose=1, save_best_only=True)

In [ ]:
import os
from scipy import ndimage, misc
image_paths = glob.glob('../data/model/key19/train/*/*')
x = []
#example(2000, 256, 256, 3)
#train_datagen.fit()
for image in image_paths:
    img = load_img(image, target_size=(256,256))
    buf = img_to_array(img)
    x.append(buf)

In [ ]:
np.shape(x)

In [ ]:
train_datagen.fit(x)
validation_datagen.fit(x)

In [ ]:
# fine-tune the model
train_history = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,
    callbacks=[checkpointer])

In [ ]:
# model.save_weights('transfer_key19_99_6p_2.h5')
print('Done')

In [ ]:
model.load_weights('../saved_weights/VGG16_key19_weights_servet.hdf5')
model.compile(loss='categorical_crossentropy', optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),metrics=['accuracy'])

In [ ]:
loss, accuracy = model.evaluate_generator(validation_generator, 
                                steps = nb_validation_samples // batch_size)
print()
print('Validation Loss:', loss)
print('Validation Accuracy', accuracy)

In [ ]:
validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=1,
    shuffle = False,
    class_mode='categorical')
val_pred_prob = model.predict_generator(validation_generator,steps = nb_validation_samples, verbose=1)
val_pred_labels = np.argmax(val_pred_prob, axis=-1)

In [ ]:
acc = 1-np.sum(val_pred_labels!=validation_generator.classes[:nb_validation_samples])/nb_validation_samples
acc

In [ ]:
idx = np.array(np.where(val_pred_labels!=validation_generator.classes))
for i in range(0,idx.shape[1]):
    print(validation_generator.filenames[idx[0,i]])

In [ ]:
test_datagen = ImageDataGenerator(preprocessing_function=subtractmean)
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=1,
    shuffle = False,
    class_mode='categorical')

In [ ]:
test_pred_prob = model.predict_generator(test_generator,steps = nb_test_samples, verbose=1)
test_pred_labels = np.argmax(test_pred_prob, axis=-1)

In [ ]:
acc = 1-np.sum(test_pred_labels!=test_generator.classes[:nb_test_samples])/nb_test_samples
acc

In [ ]:
idx = np.array(np.where(test_pred_labels!=test_generator.classes))
for i in range(0,idx.shape[1]):
    print(test_generator.filenames[idx[0,i]])

# Loading model

In [ ]:
# build the VGG16 network
base_model = applications.VGG16(weights='imagenet', include_top=False,input_shape = (img_width, img_height, 3))
print('Model loaded.')

In [ ]:
# build a classifier model to put on top of the convolutional model
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(6, activation='softmax'))

In [ ]:
# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning
top_model.load_weights(top_model_weights_path)

In [ ]:
# add the model on top of the convolutional base
model = Model(inputs= base_model.input, outputs=top_model(base_model.output))

In [ ]:
model.load_weights('../saved_weights/VGG16_key19_weights.hdf5')

In [ ]:
test_datagen = ImageDataGenerator(rescale=1. / 255)
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=1,
    shuffle = False,
    class_mode='categorical')

In [ ]:
test_pred_prob = model.predict_generator(test_generator,steps = nb_test_samples, verbose=1)
test_pred_labels = np.argmax(test_pred_prob, axis=-1)

In [ ]:
acc = 1-np.sum(test_pred_labels!=test_generator.classes[:nb_test_samples])/nb_test_samples
acc

In [ ]:
idx = np.array(np.where(test_pred_labels!=test_generator.classes))
for i in range(0,idx.shape[1]):
    print('Classified as pos {}'.format(test_pred_labels[i]+1))
    filename = '../data/model/key19/test/'+test_generator.filenames[idx[0,i]]
    plt.figure()
    image = imread(filename)
    plt.imshow(image)
    plt.title(filename)
    plt.axis('off')
    plt.show()
    


In [ ]:
model.save('VGG16.h5')

# This is for creating the model init weights

In [ ]:
def subtractmean(img):
   
    mean_image = np.mean(img)
    img -= mean_image
    img /= 128.
    

    return img

In [37]:
train_datagen = ImageDataGenerator(#preprocessing_function=subtractmean
)

validation_datagen = ImageDataGenerator(#preprocessing_function=subtractmean
)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle = False)

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    shuffle = False,
    class_mode='categorical')

epochs = 100

Found 448 images belonging to 6 classes.
Found 150 images belonging to 6 classes.


In [38]:
def save_bottlebeck_features():
    datagen = ImageDataGenerator(#preprocessing_function=subtractmean
    )

    # build the VGG16 network
    model = applications.VGG16(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    
    bottleneck_features_train = model.predict_generator(generator, nb_train_samples // batch_size, verbose=1)
    
    np.save(open('../saved_weights/bottleneck_features_train_servet.npy', 'wb'), bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    
    bottleneck_features_validation = model.predict_generator(generator, nb_validation_samples // batch_size, verbose=1)
    np.save(open('../saved_weights/bottleneck_features_validation_servet.npy', 'wb'), bottleneck_features_validation)

In [39]:
def train_top_model():
    train_data = np.load(open('../saved_weights/bottleneck_features_train_servet.npy','rb'))
    train_labels = np.array([[1,0,0,0,0,0]] * nb_train_sub_samples[0].astype(int) + 
                            [[0,1,0,0,0,0]] * nb_train_sub_samples[1].astype(int) +
                            [[0,0,1,0,0,0]] * nb_train_sub_samples[2].astype(int) + 
                            [[0,0,0,1,0,0]] * nb_train_sub_samples[3].astype(int) +
                            [[0,0,0,0,1,0]] * nb_train_sub_samples[4].astype(int) + 
                            [[0,0,0,0,0,1]] * nb_train_sub_samples[5].astype(int))
    train_labels = train_labels[0:nb_train_samples // batch_size * batch_size]

    validation_data = np.load(open('../saved_weights/bottleneck_features_validation_servet.npy','rb'))
    validation_labels = np.array([[1,0,0,0,0,0]] * nb_validation_sub_samples[0].astype(int) + 
                                 [[0,1,0,0,0,0]] * nb_validation_sub_samples[1].astype(int) +
                                 [[0,0,1,0,0,0]] * nb_validation_sub_samples[2].astype(int) + 
                                 [[0,0,0,1,0,0]] * nb_validation_sub_samples[3].astype(int) +
                                 [[0,0,0,0,1,0]] * nb_validation_sub_samples[4].astype(int) + 
                                 [[0,0,0,0,0,1]] * nb_validation_sub_samples[5].astype(int))
    validation_labels = validation_labels[0:nb_validation_samples // batch_size * batch_size]

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(6, activation='softmax'))

    model.compile(optimizer='Adam',
                  loss='categorical_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)

In [ ]:
save_bottlebeck_features()

Found 448 images belonging to 6 classes.
7/7 [==============================] - 123s    
Found 150 images belonging to 6 classes.
2/2 [==============================] - 35s     


In [ ]:
train_top_model()

Train on 448 samples, validate on 128 samples
Epoch 1/100
448/448 [==============================] - 8s - loss: 11.5423 - acc: 0.2701 - val_loss: 13.4738 - val_acc: 0.1641
Epoch 2/100
448/448 [==============================] - 6s - loss: 11.9001 - acc: 0.2612 - val_loss: 14.6070 - val_acc: 0.0938
Epoch 3/100
448/448 [==============================] - 6s - loss: 13.3478 - acc: 0.1719 - val_loss: 14.7329 - val_acc: 0.0859
Epoch 4/100
448/448 [==============================] - 6s - loss: 13.9234 - acc: 0.1362 - val_loss: 14.7329 - val_acc: 0.0859
Epoch 5/100
448/448 [==============================] - 6s - loss: 14.1033 - acc: 0.1250 - val_loss: 14.8589 - val_acc: 0.0781
Epoch 6/100
448/448 [==============================] - 6s - loss: 14.0674 - acc: 0.1272 - val_loss: 14.9848 - val_acc: 0.0703
Epoch 7/100
448/448 [==============================] - 6s - loss: 13.9954 - acc: 0.1317 - val_loss: 15.1107 - val_acc: 0.0625
Epoch 8/100
448/448 [==============================] - 6s - loss: 14.067

448/448 [==============================] - 6s - loss: 11.4769 - acc: 0.2879 - val_loss: 10.9553 - val_acc: 0.3203
Epoch 66/100
448/448 [==============================] - 6s - loss: 11.5129 - acc: 0.2857 - val_loss: 10.9553 - val_acc: 0.3203
Epoch 67/100
448/448 [==============================] - 6s - loss: 11.4410 - acc: 0.2902 - val_loss: 10.9553 - val_acc: 0.3203
Epoch 68/100
448/448 [==============================] - 6s - loss: 11.4769 - acc: 0.2879 - val_loss: 10.9553 - val_acc: 0.3203
Epoch 69/100
448/448 [==============================] - 6s - loss: 11.5129 - acc: 0.2857 - val_loss: 10.9553 - val_acc: 0.3203
Epoch 70/100
448/448 [==============================] - 6s - loss: 11.4769 - acc: 0.2879 - val_loss: 10.9553 - val_acc: 0.3203
Epoch 71/100
448/448 [==============================] - 6s - loss: 11.5489 - acc: 0.2835 - val_loss: 10.9553 - val_acc: 0.3203
Epoch 72/100
448/448 [==============================] - 6s - loss: 11.4769 - acc: 0.2879 - val_loss: 10.9553 - val_acc: 0.32